### RAG - Graphe de connaissance

#### Initialisation du graphe avec ses données

Graphe de connaissace: exemple simplifié des objets et relations typique d'un outil de SDLC/Devops (team, tasks, service, etc.)

<img src="../assets/kag.png" alt="drawing" width="400"/>

### Création du grpahe Neo4J

Un graphe de connaissance est construit à partir d'un fichier Json. Le grpahe comprend une trentaine de noeuds représentant équipe, microservice, tasks et personnes, visible sur le  [dashboard Neo4j](http://localhost:7474/browser/) (`MATCH (n) RETURN n`)

In [ ]:
import dotenv
dotenv.load_dotenv()

from langchain_community.graphs import Neo4jGraph
import json

from os import getenv

graph = Neo4jGraph(username=getenv("NEO4J_USERNAME"), password=getenv("NEO4J_PASSWORD"))

with open('../__docs__/microservices.json') as json_file:
    data = json.load(json_file)
    print(f"data: \n {data}")
    graph.query(data['query'])

#### Initialisation du graphe

Nous utilisons `langchain` pour peupler créer un vector store sur la propriété `description`des noeuds `Task`du graphe.

<img src="../assets/neo4j_graph2.png" alt="drawing" height="400"/>

In [5]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain_openai import OpenAIEmbeddings

vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    index_name='tasks',
    node_label="Task",
    text_node_properties=['name', 'description', 'status'],
    embedding_node_property='embedding',
)

#### Recherche de similarité et chat

Le vector store est maintenant disponible pour de la recherche de similarité sur la description des Task.

In [ ]:
response = vector_index.similarity_search(
    "How will RecommendationService be updated?"
)
print(response[0].page_content)

Le vecteur store associé au champ `description` des Tasks du graphe est donc accessible au modèle.

In [ ]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate 

prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", """Je dispose de données provenant d'un outil de gestion du cycle de vie logiciel (SDLC) qui incluent les informations suivantes :

Équipes de développement : noms des équipes, projets sur lesquels elles travaillent, compétences clés.
Microservices : noms des microservices, technologies utilisées, propriétaires, statut (en développement, en production, en maintenance).
Tasks : tâches assignées, priorités, deadlines, état d'avancement, équipes et personnes assignées.
Personnes : noms, rôles, compétences, historique des tâches réalisées.
J'ai besoin de synthétiser et d'obtenir des réponses précises basées sur ces données. Par exemple :

Quels sont les microservices gérés par l'équipe X ?
Qui est responsable de la tâche Y et quelle est sa deadline ?
Quelles compétences sont présentes au sein de l'équipe Z ?
Utilise les données que tu récupères pour fournir des réponses contextuelles et pertinentes aux questions liées à ces entités (équipes, microservices, tâches, personnes)."""),
        ("system", "{context}"),
        ("human", "{question}")
    ]
)
prompt_template
chain = prompt_template | ChatOpenAI() | vector_index.as_retriever()

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(), chain_type="stuff", retriever=vector_index.as_retriever())

vector_qa.invoke(
    {"query": "Comment sera mis-à-jour le service de recommandation ?"}
)

##### Les limites du RAG simple (naïf?)

Dans le cas de données très structurées comme ici, le RAG classique ("naïf") montre vite ses limites.

Le principe du RAG est de récupérer (*retrieve*) les `k` tops documents (*chunk* de nos documents originaux) du vector store possédant le plus de similarité avec notre question. 

In [ ]:
vector_qa.invoke(
    {"query": "Combien y-a-t-il de ticket Open?"}
)

Le LLM est affirmatif mais la réponse est incorrecte !

Ce n'est pas une hallucination du LLM mais une limite de notre RAG.
En effet la qualité de la réponse du LLM est directement corrélé à la qualité et quantité des informations pertinentes trouvées dans le vector store. Ici notre chaîne est paramétrée par défaut pour envoyer au LLM les 4 documents les plus pertinents.

Grâce à la requête `MATCH (t:Task) WHERE t.status = 'open' RETURN t` nous voyons que le graphe contient 5 tickets open et non pas 3. 

<img src="../assets/neo4j_graph3.png" alt="drawing" height="400"/>

#### Utiliser le LLM pour produire directement une requête sur le KG

Langchain fournit une chaîne préconfigurée `GraphCypherQAChain` avec un agent disposant d'un Tool lui permettant de générer et exécuter des requêtes sur le Knowledge Graph.

In [12]:
from langchain.chains import GraphCypherQAChain

graph.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = ChatOpenAI(temperature=0, 
    model_name='gpt-4o'),
    qa_llm = ChatOpenAI(temperature=0), 
    graph=graph, 
    verbose=True,
    allow_dangerous_requests=True
)

In [ ]:
cypher_chain.invoke(
    {"query": "Combien y-a-t-il de ticket ouverts??"}
)


Nous pouvons maintenant pleinement tirer parti de la nature structurée de nos data, par exemple en posant des questions sur des relations indirectes.

In [ ]:

cypher_chain.invoke(
    {"query": "Which services depend on Database indirectly?"}
)

#### Mixer RAG et Knowledge graph dans un Agent

Au final afin de tirer les bénéfices des 2 cas d'usage, nous pouvons construire un agent possédant ces 2 outils:
* Rag naïf - en cas de besoin de la description des tasks
* Requête sur le graphe - en cas de question sur les relations ou le dénombrage 

In [15]:
from langchain.agents import create_openai_functions_agent, Tool, AgentExecutor
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate

tools = [
    Tool(
        name="Tasks",
        func=vector_qa.invoke,
        description="""Useful when you need to answer questions about descriptions of tasks.
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Graph",
        func=cypher_chain.invoke,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

prompt = prompt = ChatPromptTemplate.from_messages([
  ("system", "You are a helpful assistant"),
  ("placeholder", "{chat_history}"),
  ("human", "{input}"),
  ("placeholder", "{agent_scratchpad}")]
)
agent = create_openai_functions_agent(
    ChatOpenAI(temperature=0, model_name='gpt-4o'), tools, prompt
)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [ ]:
response = agent_executor.invoke({"input": "Which team is assigned to maintain PaymentService?"})

In [ ]:
response = agent_executor.invoke({"input": "Which tasks have optimization in their description?"})